This notebook exists to help perform some transformations and inspections of data.  Goal is to eventually collapse this into a .py file.

Data Sources / credits:
* Census data selector CSV download from: https://www.census.gov/data-tools/demo/saipe/#/?map_geoSelector=aa_c&s_year=2018,2017,2015,2009&map_yearSelector=2009
* Andrew's API key: 809a11b265cd8f07473e64706d722d9c5aa77b23


In [1]:
# Install some helpful libraries
#!pip install geopy

# Importing necessary libraries
import pandas as pd
from geopy.geocoders import Nominatim
from functools import partial

# Setup our geolocator
geolocator = Nominatim(user_agent="college-town-weather-extractor-v2")
geocode = partial(geolocator.geocode, language="en", timeout=120)

In [6]:
# Reading the files
fpath = '../economics-input.csv'
city_df = pd.read_csv(fpath,header=0)
city_df['county'],city_df['state'],city_df['poverty_pct'],city_df['med_income']='','','',''
fpath = './census/SAIPESNC_30MAY20_00_49_30_76.csv'
census_df = pd.read_csv(fpath)

In [ ]:
# Identify counties and econ data for each city
# Starting with a single year
year = 2018

for i in range(city_df.shape[0]):
    city,state = city_df['Geographic Location'][i].split(",")
    location = geolocator.geocode(city+state+" usa")
    loc = location[0].split(',')[1:3]
    #print(city,state," - ",end="")
    #print(location)
    cty = ""
    for l in location[0].split(','):
        if 'county' in l.lower():
            cty = l
        else:
            if 'parish' in l.lower():
                cty = l
    if cty == "":
        print("County not found for:",location)
        continue
    city_df.at[i,'county'],city_df.at[i,'state']=cty,state
    for n in range(census_df.shape[0]):
        if census_df['State / County Name'][n].split("(")[0].strip() == cty.strip():
            if int(census_df['Year'][n]) == year:
                city_df.at[i,'poverty_pct'],city_df['med_income'] = float(census_df['All Ages in Poverty Percent'][n]),census_df['Median Household Income in Dollars'][n]

County not found for: Chestnut Hill, Massachusetts, 02102, United States of America
County not found for: Hampton, Hampton City, Virginia, 23669-4019, United States of America
County not found for: Harrisonburg, Virginia, 22801:22802, United States of America
County not found for: Lynchburg, Lynchburg (city), Virginia, 24505, United States of America
County not found for: Baltimore, Maryland, United States of America
County not found for: Norfolk, Virginia, United States of America
County not found for: Cedar Falls, Iowa, 50613, United States of America
County not found for: Norman, Oklahoma, United States of America
County not found for: Stillwater, Oklahoma, United States of America
County not found for: Stillwater, Oklahoma, United States of America
County not found for: Norfolk, Virginia, United States of America
County not found for: Richmond, Richmond City, Virginia, 23298, United States of America
County not found for: Charlottesville, Virginia, United States of America


In [ ]:
city_df

In [ ]:
census_df

In [6]:
city_df.to_csv('../economics-output.csv', index=True)